## The first cell will only execute if you're using Google Colab AND have not cloned the repository yet

In [1]:
import os
import sys
import subprocess

# Check if running in Google Colab
colab_setup = "google.colab" in sys.modules

repo_url = "https://github.com/sprouse9/URI_CapstoneProject.git"
repo_name = "URI_CapstoneProject"

# Clone only if running in Google Colab. Prevent cloning repo if already cloned.
if colab_setup and not os.path.exists('../' + repo_name):
    # Clone the repository if not already cloned

    if not os.path.exists(repo_name):
        print(f"Cloning repository: {repo_url}...")
        !git clone {repo_url}

    # Change directory to the repository
    %cd {repo_name}

Cloning repository: https://github.com/sprouse9/URI_CapstoneProject.git...
Cloning into 'URI_CapstoneProject'...
remote: Enumerating objects: 1455, done.
remote: Counting objects: 100% (382/382), done.
remote: Compressing objects: 100% (334/334), done.
remote: Total 1455 (delta 40), reused 355 (delta 23), pack-reused 1073 (from 2)
Receiving objects: 100% (1455/1455), 38.02 MiB | 34.51 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/URI_CapstoneProject


#### This next cell takes care of the dataset download from my Google Drive as a zip file.  
#### The zip file will be auto extracted to your local machine or instance of Colab.
#### The download will not occur if the zip file or the extracted folder already exists.
#### The data folder 'archive' will not be unzipped again if already exists

In [3]:
import zipfile
import importlib.util

zip_filename = "CarDetectionDataSet.zip"
extract_folder = "archive"  # Define the folder where files are extracted

print(f"Working dir: {os.getcwd()}")  # This shows your current working directory

# Download only if the zip file and extracted folder don't exist
if not os.path.exists(zip_filename) and not os.path.exists(extract_folder):
    # Check if gdown is installed before attempting to install
    if importlib.util.find_spec("gdown") is None:
        print("gdown not found. Installing...")
        !pip install gdown

    print(f"{zip_filename} not found. Downloading...")
    !gdown 1JFAfrbUfXtiF-xwko2ACB-snDwIsj31h -O {zip_filename}
else:
    print(f"Skipping download. {zip_filename} or {extract_folder} already exists.")

# Extract only if the extracted folder does not exist
if not os.path.exists(extract_folder):
    print(f"Extracting {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall()
    print(f"Extraction complete: {extract_folder}")
else:
    print(f"{extract_folder} already exists. Skipping extraction.")


Working dir: /content/URI_CapstoneProject
Skipping download. CarDetectionDataSet.zip or archive already exists.
Extracting CarDetectionDataSet.zip...
Extraction complete: archive


## The data folder has been setup.

In [4]:
if importlib.util.find_spec("ultralytics") is None:
    print("gdown not found. Installing...")
    !pip install -q ultralytics

if importlib.util.find_spec("torch") is None:
    print("torch not found. Installing...")
    !pip install -q torch

import torch
print("PyTorch Version:", torch.__version__)

gdown not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
PyTorch Version: 2.5.1+cu124


In [5]:
# take care of library related path issues regardless of the
# Operating system used

import yaml

# Load the existing dataset.yaml configuration
with open('dataset.yaml', 'r') as f:
    dataset = yaml.safe_load(f)

# Get the current working directory
cwd = os.getcwd()

# Dynamically adjust the 'path'
# This sets an absolute path for ultralytics so that it resolves correctly
# Shouldn't be necessary but here we are
dataset['path'] = os.path.join(cwd, 'archive')

print("Resolved dataset path:", dataset['path'])

with open('dataset_updated.yaml', 'w') as f:
    yaml.dump(dataset, f)

Resolved dataset path: /content/URI_CapstoneProject/archive


In [7]:
# Get GPU name and VRAM

# Set the device: use "cuda" if available, otherwise "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Optionally, get VRAM information if using a GPU
if device == "cuda":
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    gpu_name = torch.cuda.get_device_name(0)
    print(f"Using GPU ({gpu_name}) with {vram:.2f} GB VRAM")
else:
    vram = None
    print("Using CPU")

#print("GPU Name:", gpu_name)
print("GPU VRAM:", torch.cuda.get_device_properties(0).total_memory / (1024**3), "GiB")
print(f"Available VRAM: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

Using GPU (Tesla T4) with 15.83 GB VRAM
GPU VRAM: 14.74127197265625 GiB
Available VRAM: 0.00 GB


In [8]:
import pandas as pd
from ultralytics import YOLO

if device == "cuda":
    learning_rates = [0.0005, 0.002, 0.01]
    epochs_list = [10, 20, 50]
    image_sz = 640
    # Using a tiered approach based on available VRAM:
    if vram >= 15:
        print("VRAM is very large")
        batch_sz = 64
    elif vram > 10:
        print("VRAM is large")
        batch_sz = 32  # a moderate increase
    else:
        batch_sz = 16
else:
    learning_rates = [0.0005, 0.002]
    epochs_list = [10]
    image_sz = 320
    batch_sz = 4

# List to store results from each experiment
results_summary = []

for lr in learning_rates:
    for epochs in epochs_list:
        print(f"Training with lr: {lr}, epochs: {epochs}")

        # Conditionally clear CUDA memory if available
        if torch.cuda.is_available():
          torch.cuda.empty_cache()
          torch.cuda.ipc_collect()
        else:
          print("CUDA not available; skipping CUDA memory cleanup.")

        # Initialize the model
        model = YOLO("yolov8n.pt")

        # Train the model with the current hyperparameters
        results = model.train(
            data="dataset_updated.yaml",
            epochs=epochs,
            batch=batch_sz,
            imgsz=image_sz,
            lr0=lr,
            #cache=True,
            cache='disk',
            optimizer="AdamW",
            project=f"runs/train/lr{lr}_ep{epochs}"
        )

        # Get the results dictionary
        rdict = results.results_dict  # Contains keys like 'metrics/precision(B)' etc.

        # Append the metrics along with the hyperparameters to our list
        results_summary.append({
            "lr": lr,
            "epochs": epochs,
            "precision": rdict.get("metrics/precision(B)", None),
            "recall": rdict.get("metrics/recall(B)", None),
            "mAP50": rdict.get("metrics/mAP50(B)", None),
            "mAP50-95": rdict.get("metrics/mAP50-95(B)", None),
            "fitness": rdict.get("fitness", None)
        })


        # Conditionally clear CUDA memory if available
        # if torch.cuda.is_available():
        #    torch.cuda.empty_cache()
        #    torch.cuda.ipc_collect()
        # else:
        #     print("CUDA not available; skipping CUDA memory cleanup.")


# Convert the results list into a DataFrame for easy viewing
results_df = pd.DataFrame(results_summary)
print("\nBaseline Metrics for Each Test:")
print(results_df)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
VRAM is very large
Training with lr: 0.0005, epochs: 10


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.2MB/s]


Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_updated.yaml, epochs=10, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=disk, device=None, workers=8, project=runs/train/lr0.0005_ep10, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 22.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50      7.65G     0.3746     0.2338     0.7804        174        640: 100%|██████████| 7/7 [00:04<00:00,  1.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.18s/it]

                   all         99        644      0.949      0.769      0.928      0.777



50 epochs completed in 0.129 hours.
Optimizer stripped from runs/train/lr0.01_ep50/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/train/lr0.01_ep50/train/weights/best.pt, 6.2MB

Validating runs/train/lr0.01_ep50/train/weights/best.pt...
Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


                   all         99        644       0.95      0.769      0.927      0.777
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/train/lr0.01_ep50/train

Baseline Metrics for Each Test:
       lr  epochs  precision    recall     mAP50  mAP50-95   fitness
0  0.0005      10   0.928012  0.562112  0.821413  0.609509  0.630699
1  0.0005      20   0.849446  0.784161  0.892621  0.694569  0.714374
2  0.0005      50   0.829223  0.821429  0.907031  0.726589  0.744633
3  0.0020      10   0.945445  0.618930  0.849143  0.664448  0.682917
4  0.0020      20   0.832686  0.803705  0.895060  0.718127  0.735820
5  0.0020      50   0.854076  0.808862  0.909397  0.750816  0.766674
6  0.0100      10   0.870470  0.739130  0.862304  0.596275  0.622878
7  0.0100      20   0.917089  0.754658  0.879957  0.705703  0.723128
8  0.0100      50   0.949638  0.768634  0.927499  0.777226  0.792253
